In [3]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [4]:
class HappyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        data = self.data.iloc[idx, 0:].values
        data = torch.Tensor(data.astype('float'))
        if self.transform:
            data = self.transform(data)
        inputs = data[0:2]
        labels = data[2:4]
        return inputs, labels

happy = HappyDataset(csv_file='happy.csv')

dataloader = DataLoader(happy, batch_size=4,
                        shuffle=False, num_workers=1)

In [5]:
class float_Net(nn.Module):
    def __init__(self):
        super(float_Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = float_Net()
net.cuda(0)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for i in range(500):
    tLoss = 0
    for index, data in enumerate(dataloader):
        inputs, labels = data
        inputs = inputs.cuda(0)
        labels = labels.cuda(0)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        tLoss += loss.item()
    if i%100 == 0:
        print tLoss

1.35132692009
1.0141608417
0.95705665648


Process Process-234:
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
  File "/usr/lib/python2.7/multiprocessing/queues.py", line 131, in get
    if timeout < 0 or not self._poll(timeout):
KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 500, i

IndexError: string index out of range

In [11]:
print net(torch.Tensor([0,0]).cuda(0)).data.cpu()
print net(torch.Tensor([0,1]).cuda(0)).data.cpu()
print net(torch.Tensor([1,0]).cuda(0)).data.cpu()
print net(torch.Tensor([1,1]).cuda(0)).data.cpu()

tensor([0.0403])
tensor([0.9818])
tensor([0.9462])
tensor([0.0521])


In [6]:
class oSample(torch.autograd.Function):
    def forward(ctx, inputs):
        delt = 1*pow(2,-5)
        M = (inputs/delt).round()
        M[M>=128] = 127
        M[M<-127] = -128
        S = delt*M
        return S
    def backward(ctx, g):
        return g

def sampleStateDict(net):
    Dict = net.state_dict()
    Key = Dict.keys()
    for i in Key:
        Dict[i] = oSample()(Dict[i])
    net.load_state_dict(Dict)

def protectStateDict(net):
    Dict = net.state_dict()
    Key = Dict.keys()
    for i in Key:
        Dict[i] = Dict[i]*1
    return Dict

In [7]:
class sample_Net(nn.Module):
    def __init__(self):
        super(sample_Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)
        self.oSample = oSample()

    def forward(self, x):
        
        x = oSample()(F.relu(oSample()(self.fc1(x))))
        x = oSample()(self.fc2(x))
        return x

sample_net = sample_Net()
sample_net.cuda(0)
#sample_net.load_state_dict(lol)

sample_Net(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [8]:
print sample_net(torch.Tensor([0,0]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([0,1]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([1,0]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([1,1]).cuda(0)).data.cpu()

tensor([-0.0625])
tensor([0.1562])
tensor([-0.3125])
tensor([-0.0938])


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(sample_net.parameters(), lr=0.01)
for i in range(1000):
    tLoss = 0
    for index, data in enumerate(dataloader):
        inputs, labels = data
        inputs = inputs.cuda(0)
        labels = labels.cuda(0)
        optimizer.zero_grad()
        TMP = protectStateDict(sample_net)
        sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        optimizer.step()
        tLoss += loss.item()
    if tLoss == 0:
        break
    if i%100 == 5:
        print tLoss
    #sampleStateDict(sample_net)
TMP = protectStateDict(sample_net)
sampleStateDict(sample_net)
print sample_net(torch.Tensor([0,0]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([0,1]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([1,0]).cuda(0)).data.cpu()
print sample_net(torch.Tensor([1,1]).cuda(0)).data.cpu()
sample_net.load_state_dict(TMP)

0.515869140625
0.343994140625
0.33349609375
